<a href="https://colab.research.google.com/github/Amith-kuppili/AI-for-frontend/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from typing import Text
import tensorflow as tf

Text_file=tf.keras.utils.get_file(
    fname='fra-eng.zip',
    origin='http://storage.googleapis.com/download.tensorflow.org/data/fra-eng.zip',
    extract=True
)
print('Text_file')

3423204/3423204 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Text_file


In [2]:
import pathlib
extracted_dir = pathlib.Path(Text_file).parent
Text_file = extracted_dir / 'fra-eng_extracted' / 'fra.txt'
print(Text_file)

/root/.keras/datasets/fra-eng_extracted/fra.txt


In [3]:
with open(Text_file) as fp:
    test_pair=[line for line in fp]

In [4]:
import os
print(os.listdir('/root/.keras/datasets/'))

['fra-eng.zip', 'fra-eng_extracted']


In [5]:
import os
print(os.listdir('/root/.keras/datasets/fra-eng_extracted/'))

['_about.txt', 'fra.txt']


In [6]:
import random
for _ in range(5):
    print(random.choice(test_pair))

We need to do this right.	On doit faire ça bien.

Give me an example.	Fournissez-moi un exemple.

I used to love that.	J'adorais ça.

Despite everything, Tom started to relax.	Malgré tout, Tom commença à se détendre.

I read a book while eating.	J'ai lu un livre en mangeant.



In [7]:
import unicodedata
import re

def normalize(text):
  # Split the line into English and French parts first
  eng, fre = text.strip().lower().split('\t')

  # Normalize and clean the English part
  eng = unicodedata.normalize('NFKC', eng)
  eng = re.sub(r'([,.!?])', r' \1', eng)
  eng = re.sub(r'[^a-zA-Z,.!?]+', ' ', eng)
  eng = re.sub(r'\s+', ' ', eng).strip()

  # Normalize and clean the French part
  fre = unicodedata.normalize('NFKC', fre)
  fre = re.sub(r'([,.!?])', r' \1', fre)
  fre = re.sub(r'[^a-zA-Z,.!?]+', ' ', fre)
  fre = re.sub(r'\s+', ' ', fre).strip()

  # Add start and end tokens to the French part
  fre = '[start] ' + fre + ' [end]'

  return eng, fre

In [8]:
with open(Text_file) as fp:
    test_pair=[normalize(line) for line in fp]

In [9]:
for _ in range(5):
  print(random.choice(test_pair))

('i seldom eat dairy products .', '[start] je mange rarement de produits laitiers . [end]')
('don t mind her .', '[start] ne te soucie pas d elle . [end]')
('i cannot look at this photo without being reminded of my school days .', '[start] je ne peux pas regarder cette photo sans que cela me rappelle l poque o j allais l cole . [end]')
('i really want to see tom today .', '[start] je veux vraiment voir tom aujourd hui . [end]')
('i would like a cup of coffee .', '[start] j appr cierais une tasse de caf . [end]')


In [10]:
eng_tokens_set, fre_tokens_set = set(), set()
eng_maxlen, fre_maxlen = 0, 0

for eng, fre in test_pair:
  eng_tokens = eng.split()
  fre_tokens = fre.split()

  eng_tokens_set.update(eng_tokens)
  fre_tokens_set.update(fre_tokens)

  eng_maxlen = max(eng_maxlen, len(eng_tokens))
  fre_maxlen = max(fre_maxlen, len(fre_tokens))

print(f"Number of unique English tokens: {len(eng_tokens_set)}")
print(f"Number of unique French tokens: {len(fre_tokens_set)}")
print(f"Maximum English sentence length: {eng_maxlen}")
print(f"Maximum French sentence length: {fre_maxlen}")

Number of unique English tokens: 13749
Number of unique French tokens: 19770
Maximum English sentence length: 52
Maximum French sentence length: 68


In [14]:
import pickle
with open('text_pairs.pickle','wb') as fp:
  pickle.dump(test_pair,fp)

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import pickle

# Load preprocessed text pairs
with open("text_pairs.pickle", 'rb') as fp:
    test_pair = pickle.load(fp)

In [20]:
# Shuffle the data
import random
random.shuffle(test_pair)

# Split into train and test sets
n_val = int(0.15 * len(test_pair))
n_train = len(test_pair) - 2 * n_val
train_pair = test_pair[:n_train]
val_pair = test_pair[n_train: n_train + n_val]
test_pair = test_pair[n_train + n_val:]

In [21]:
# Vocabulary sizes and sequence length
vocab_en = 10000
vocab_fr = 20000
seq_length = 25

# Initialize TextVectorization layers
eng_vect = TextVectorization(
    max_tokens=vocab_en,
    standardize=None,
    split='whitespace',
    output_mode='int',
    output_sequence_length=seq_length
)

fre_vect = TextVectorization(
    max_tokens=vocab_fr,
    standardize=None,
    split='whitespace',
    output_mode='int',
    output_sequence_length=seq_length + 1  # +1 for start token
)

# Adapt TextVectorization layers to training data
train_eng = [pair[0] for pair in train_pair]
train_fre = [pair[1] for pair in train_pair]

eng_vect.adapt(train_eng)
fre_vect.adapt(train_fre)

# Serialize the vectorization layers and training/test data
with open('vectorize.pickle', 'wb') as fp:
    data = {'train': train_pair,
            'test': test_pair,
            'eng_vect': eng_vect.get_config(),
            'fre_vect': fre_vect.get_config(),
            'eng_weights': eng_vect.get_weights(),
            'fre_weights': fre_vect.get_weights()
            }
    pickle.dump(data, fp)

# Load serialized data
with open("vectorize.pickle", 'rb') as fp:
    data = pickle.load(fp)

# Retrieve train and test pairs
train_pair = data['train']
test_pair = data['test']

# Reconstruct TextVectorization layers
eng_vect = TextVectorization.from_config(data['eng_vect'])
eng_vect.set_weights(data['eng_weights'])
fre_vect = TextVectorization.from_config(data['fre_vect'])
fre_vect.set_weights(data['fre_weights'])

# Define function to format dataset
def format_dataset(eng, fre):
    eng = eng_vect(eng)
    fre = fre_vect(fre)
    source = {'encode_inp': eng,
              'decode_inp': fre[:, :-1]
              }
    target = fre[:, 1:]
    return (source, target)

# Define function to create dataset
def make_dataset(pairs, batchsize=64):
    eng_text, fre_text = zip(*pairs)
    dataset = tf.data.Dataset.from_tensor_slices((list(eng_text), list(fre_text)))
    return dataset.shuffle(2048).batch(batchsize).map(format_dataset).prefetch(16)#.cache()

# Create TensorFlow datasets for training and testing
train_ds = make_dataset(train_pair)
test_ds = make_dataset(test_pair)

**Positional Embeddings**

In [24]:
import numpy as np

# Function to generate positional encoding matrix
def pos_enc_matrix(L, d, n=10000):
    assert d % 2 == 0
    d2 = d // 2

    P = np.zeros((L, d))
    k = np.arange(L).reshape(-1, 1)
    i = np.arange(d2).reshape(1, -1)

    denom = np.power(n, -i / d2)
    args = k * denom

    P[:, ::2] = np.sin(args)
    P[:, 1::2] = np.cos(args)
    return P

# Custom Keras layer for positional embedding
class PositionalEmbedding(tf.keras.layers.Layer):

    def __init__(self, seq_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.seq_length = seq_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

        self.token_embeddings = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)
        matrix = pos_enc_matrix(seq_length, embed_dim)

        self.positional_embedding = tf.constant(matrix, dtype='float32')

    def call(self, inputs):
        embedded_tokens = self.token_embeddings(inputs)
        return embedded_tokens + self.positional_embedding

    def compute_mask(self, *args, **kwargs):
        return self.token_embeddings.compute_mask(*args, **kwargs)

    def get_config(self):
        config = super().get_config()
        config.update({
            'seq_length': self.seq_length,
            'vocab_size': self.vocab_size,
            'embed_dim': self.embed_dim
        })

# Usage and Validation
vocab_en = 10000
seq_length = 25

for inputs, targets in train_ds.take(1):
    embed_en = PositionalEmbedding(seq_length, vocab_en, embed_dim=512)
    en_emb = embed_en(inputs['encode_inp'])
    print(en_emb._keras_mask)

FailedPreconditionError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:3 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::MemoryCacheImpl::Prefetch::Map: Table not initialized.
	 [[{{node text_vectorization_1_2/None_Lookup/LookupTableFindV2}}]] [Op:IteratorGetNext] name: 